In [ ]:
!pip install yfinance


In [ ]:
import yfinance as yf
import pandas as pd

def get_fo_stock_data(symbols, start_date, end_date):
    """
    Get OHLC data for F&O listed stocks in India.

    Parameters:
        symbols (list): List of stock symbols (e.g., ['TCS.NS', 'RELIANCE.NS', ...]).
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.

    Returns:
        pd.DataFrame: A DataFrame containing OHLC data for the specified stocks and date range.
    """
    # Download OHLC data using yfinance
    data = yf.download(symbols, start=start_date, end=end_date)

    # Keep only the 'Adj Close' prices and drop any rows with missing values
    data = data['Adj Close'].dropna()

    return data


# Example usage:
symbols_list = ['TCS.NS', 'RELIANCE.NS', 'INFY.NS', 'HDFCBANK.NS', 'HDFC.NS']  # Add more symbols as needed
start_date = '2020-01-01'
end_date = '2023-01-01'

ohlc_data = get_fo_stock_data(symbols_list, start_date, end_date)
print(ohlc_data.head())


[*********************100%***********************]  5 of 5 completed
                HDFC.NS  HDFCBANK.NS     INFY.NS  RELIANCE.NS       TCS.NS
Date                                                                      
2020-01-01  2292.563965  1244.189697  672.164490  1480.595825  2019.016968
2020-01-02  2323.129395  1252.120361  670.203247  1505.802124  2009.748413
2020-01-03  2311.873291  1234.264160  680.511292  1507.616577  2049.801514
2020-01-06  2245.610352  1207.552856  673.988770  1472.651367  2049.614990
2020-01-07  2274.762207  1226.674072  664.000183  1495.307617  2054.644531


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Step 1 and Step 2 code (data acquisition and library imports) go here...

def get_top_5_performers(data):
    """
    Calculate 52-week rolling returns for each stock and select the top 5 performers.

    Parameters:
        data (pd.DataFrame): DataFrame containing OHLC data for F&O listed stocks.

    Returns:
        pd.DataFrame: DataFrame containing the top 5 performing stocks' 52-week rolling returns.
    """
    # Calculate the percentage change in adjusted closing prices
    returns = data.pct_change()

    # Calculate 52-week rolling returns for each stock
    rolling_returns = returns.rolling(window=52).apply(lambda x: np.prod(1 + x) - 1)

    # Get the latest date's rolling returns
    latest_rolling_returns = rolling_returns.iloc[-1]

    # Select the top 5 performers
    top_5_performers = latest_rolling_returns.nlargest(5)

    return top_5_performers

# Example usage:
# Assuming you have already obtained the OHLC data as 'ohlc_data'
top_5_performers = get_top_5_performers(ohlc_data)
print(top_5_performers)


HDFCBANK.NS    0.125540
HDFC.NS        0.122741
RELIANCE.NS    0.057478
TCS.NS         0.046582
INFY.NS        0.023311
Name: 2022-12-30 00:00:00, dtype: float64


In [ ]:
import requests

def get_current_stock_price(stock):
    url = "https://twelve-data1.p.rapidapi.com/stocks"
    querystring = {"exchange": "NASDAQ", "format": "json", "symbol": stock}

    headers = {
        "X-RapidAPI-Key": "d3eee0660cmshbdf6933fa97c3c2p1e906ejsna482c89a4b2d",
        "X-RapidAPI-Host": "twelve-data1.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=querystring)
        data = response.json()
        current_price = data[0]['price'] if data else None
        return current_price
    except Exception as e:
        print(f"Error fetching stock price for {stock}: {e}")
        return None

# Rest of the code remains the same as before

# ...

# In the __main__ block:
if __name__ == "__main__":
    stock_symbol = 'AAPL'  # Replace this with the stock symbol you want to fetch the price for
    current_price = get_current_stock_price(stock_symbol)
    if current_price is not None:
        print(f"The current price of {stock_symbol} is ${current_price:.2f}")
    else:
        print(f"Unable to fetch the current price for {stock_symbol}")


Error fetching stock price for AAPL: 0
Unable to fetch the current price for AAPL


In [ ]:
def buy_stock(stock, allocation, portfolio):
    # Get the current price of the stock
    current_price = get_current_stock_price(stock)

    # Calculate the quantity to buy based on the allocation and the current price
    available_cash = 10000  # Assuming you have $10,000 available to invest
    quantity_to_buy = int((available_cash * allocation) / current_price)

    # Check if there is enough cash to buy the stock
    if quantity_to_buy * current_price > available_cash:
        print(f"Not enough cash to buy {stock}.")
        return

    # Update the portfolio with the purchased quantity
    if stock in portfolio:
        portfolio[stock] += quantity_to_buy
    else:
        portfolio[stock] = quantity_to_buy

    # Deduct the purchased amount from available cash
    available_cash -= quantity_to_buy * current_price

    print(f"Bought {quantity_to_buy} shares of {stock} at ${current_price:.2f} per share.")
    print(f"Remaining cash: ${available_cash:.2f}")


In [ ]:
def buy_stocks(top_performers, portfolio):
    # 'portfolio' is a dictionary that tracks the current holdings and quantities of stocks

    # Assuming each stock has an equal allocation in the portfolio
    allocation_per_stock = 1.0 / len(top_performers)

    for stock in top_performers:
        # Assuming you have a function to buy a stock with a specific allocation and update the portfolio
        # Implement the buy_stock() function according to your trading platform or data structure
        buy_stock(stock, allocation_per_stock, portfolio)


In [ ]:
def set_stop_loss(portfolio, stop_loss_percentage):
    for stock, quantity in portfolio.items():
        # Get the current price of the stock (you need an appropriate function for this)
        current_price = get_current_stock_price(stock)

        # Calculate the stop-loss price
        stop_loss_price = current_price * (1 - stop_loss_percentage)

        # Assuming you have a function to set a stop-loss level for a specific stock
        # Implement the set_stop_loss() function according to your trading platform or data structure
        set_stop_loss(stock, stop_loss_price)


In [71]:
def weekly_update():
    top_performers = get_top_performers()
    buy_stocks(top_performers, portfolio)
    set_stop_loss(portfolio, stop_loss_percentage)
    # Other actions or checks you want to perform weekly


In [ ]:
!pip install pyfolio
!pip install empyrical


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.1 MB/s eta 0:00:00
  Created wheel for pyfolio: filename=pyfolio-0.9.2-py3-none-any.whl size=88663 sha256=78411fb48b2bfff172123a63ee01be4cfba8342a147c2aeecc2da23173dc3512
  Stored in directory: /root/.cache/pip/wheels/71/38/bc/e53700cfd8b0ad6b539d2fbaaf060ed8a299e7622a5b86ef42
  Created wheel for empyrical: filename=empyrical-0.5.5-py3-none-any.whl size=39762 sha256=cd33ee949ee3a779563b793c9077e32a1a6cb8c4f6f569b0106bbacfa259a3e3
  Stored in directory: /root/.cache/pip/wheels/0e/2e/f2/d6d2d9a1eb8fbbd9949bb5d4c00f753e3b74e5bd7ed10b1d36
Successfully built pyfolio empyrical


In [ ]:
import pandas as pd
import numpy as np
import pyfolio as pf

def get_stock_data():
    # Function to fetch historical stock data for the stocks in the portfolio
    # Replace this with your data retrieval mechanism (e.g., API or local CSV file)
    # Returns a pandas DataFrame with columns: 'Date', 'StockSymbol', 'AdjClose'
    pass

def get_portfolio_returns(portfolio, stock_data):
    # Function to calculate daily portfolio returns based on the stock holdings and data
    # Returns a pandas DataFrame with columns: 'Date', 'Return'
    pass


/usr/local/lib/python3.10/dist-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [ ]:
def backtest_momentum_strategy():
    # Get historical stock data
    stock_data = get_stock_data()

    # Get the portfolio holdings and their allocations based on the momentum strategy
    top_performers = get_top_performers()
    portfolio = {stock: 1.0 / len(top_performers) for stock in top_performers}

    # Calculate daily portfolio returns
    portfolio_returns = get_portfolio_returns(portfolio, stock_data)
    return portfolio_returns


In [ ]:
def analyze_backtest(portfolio_returns):
    # Convert daily portfolio returns to a pandas Series
    portfolio_returns_series = pd.Series(portfolio_returns['Return'].values,
                                         index=pd.to_datetime(portfolio_returns['Date']))

    # Calculate performance metrics using Pyfolio
    pf.create_returns_tear_sheet(portfolio_returns_series)


In [ ]:
import pandas as pd
import numpy as np

# Sample function to generate random historical stock data
def get_stock_data():
    np.random.seed(0)
    date_range = pd.date_range(start='2023-01-01', end='2023-12-31', freq='B')
    symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'FB']
    data = {
        'Date': date_range,
        'StockSymbol': np.random.choice(symbols, len(date_range)),
        'AdjClose': np.random.randint(100, 300, len(date_range))
    }
    return pd.DataFrame(data)

def get_top_performers(stock_data, num_top_performers=5):
    # Calculate the 52-week rolling returns for each stock
    stock_data['RollingReturn'] = stock_data.groupby('StockSymbol')['AdjClose'].pct_change().rolling(window=252).sum()

    # Get the top performers based on 52-week rolling returns
    top_performers = stock_data.groupby('StockSymbol')['RollingReturn'].last().nlargest(num_top_performers)

    return top_performers.index.tolist()

# Rest of the code remains the same as before

# ...

# In the __main__ block:
if __name__ == "__main__":
    # Generate random historical stock data
    stock_data = get_stock_data()

    # Get the top 5 performers based on 52-week rolling returns
    top_performers = get_top_performers(stock_data)
    print("Top performers:", top_performers)


Top performers: ['AMZN', 'AAPL', 'FB', 'GOOGL', 'MSFT']
